In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

# External Datasets

Load in datasets for population, urbanization and gdp values for the US and Canada.

In [2]:
population = pd.read_csv('population.csv')
urbanization = pd.read_csv('urbanization.csv',index_col=0)
gdp = pd.read_csv('gdp.csv')

# Internal Datasets

Load in the mapper dataset, and our data.

In [3]:
mapper = pd.read_excel('Mapping material names_20210324.xlsx',header=2,usecols='B:U').replace(r'\n','', regex=True) 

In [4]:
df = pd.read_excel('../Dataset/dataset.xlsx',header=1,usecols='B:BKX')

# Additional Categories

Specify the mapping from additional category codes to their categories, and also list the columns which are not materials.

In [5]:
additional_categories_map = {v:k for k,v in {
    'Continuous Footings':'0CF',
    'Foundation Walls':'0FW',
    'Spread Footings':'0SF',
    'Column Piers':'0CP',
    'Columns Supporting Floors':'CSF',
    'Floor Girders and Beams':'FGB',
    'Floor Trusses':'0FT',
    'Floor Joists':'0FJ',
    'Columns Supporting Roofs':'CSR',
    'Roof Girders and Beams':'RGB',
    'Roof Trusses':'0RT',
    'Roof Joists':'0RJ',
    'Parking Bumpers':'0PB',
    'Precast Concrete Stair Treads':'PCS',
    'Roof Curbs':'0RC',
    'Exterior Wall Construction':'EWC',
    'Composite Decking':'CPD',
    'Cast-in-Place concrete':'CIC',
    'Floor Structural Frame':'FSF',
    'Associated Metal Fabrications':'AMF',
    'Floor Construction Supplementary Components':'FCS',
    'Roof Construction Supplementary Components':'RCS',
    'Residential Elevators':'0RE',
    'Vegetated Low-Slope Roofing':'VLR',
    'Swimming Pools':'SWP',
    'Excavation Soil Anchors':'ESA',
    'Floor Trusses':'FTS',
    'Roof Window and Skylight Performance':'RWS'}.items()
}

additional_categories_map['0FT'] = 'Floor Trusses'

headings = ['Building Identifier',
            'Country',
 'City',
 'Quality / Stage of Data',
 'Construction Date',
 'Building Type',
 'Contributor']

# Get Heeren & Fishman (2019) Material Names

Using the Mapping Names datasheet, find the relevant entry and identify the name.

In [10]:
def get_material_name(l):
    split = re.split('[_\.\ ]',l) #Split up the code into its requisite parts
    result = mapper[mapper['Unnamed: 7'] == split[1]+'.'+split[2]] #Filter by Level 4 Master Format
    if len(result) == 0:
        result = mapper #If that code does not exist in the table, reset
    if len(result) == 1:
        return result['Mapping Table'].values[0] #If it maps to exactly one value, return that. We do this check after every step
    if split[3] != '000': #Check if there is an additional code, and if so filter by that
        result = result[result['Level 5\n'] == additional_categories_map[split[3]]]
        if len(result) == 1:
            return result['Mapping Table'].values[0]
        
    #Now filter by UniFormat.
    #Filter only by the level of UniFormat present. If the code is XX 00 00, for example, then we only have Level 1.
    if int(split[5]) == 0: 
        result = result[result['Unnamed: 12'] == f'{split[4]} 00 00']
        if len(result) == 1:
            return result['Mapping Table'].values[0]
    elif int(split[6]) == 0:
        result = result[(result['Unnamed: 14'] == f'{split[4]} {split[5]} 00') | (result['Unnamed: 16'] == f'{split[4]} {split[5]} 00')]
        if len(result) == 1:
            return result['Mapping Table'].values[0]
    else:
        result = result[result['Unnamed: 18'] == f'{split[4]} {split[5]} {split[6]}']
        if len(result) == 1:
            return result['Mapping Table'].values[0]
        
    #If we couldn't find it, or there is an unspecified edge case, return None. 
    if len(result) == 0:
        return None
    
    #If there are multiple results but they all map to the same material, return that material.
    if all(element == result['Mapping Table'].values[0] for element in result['Mapping Table'].values):
        return result['Mapping Table'].values[0]
    else:
        return None

# Get external data from datasheets

The following functions obtain data from the external datasheets, if available.

In [11]:
def get_gdp(v):
    if v['Country'] == 'CA':
        idx = 33
    else:
        idx = 249
    if v['Construction Date'] > 2019:
        return gdp.at[idx,'2019']
    if v['Construction Date'] < 1960:
        return None
    return gdp.at[idx,str(int(v['Construction Date']))]

In [12]:
def get_urbanization(v):
    try:
        range_low = int(np.floor(v['Construction Date']/5)*5)
        range_high = int(range_low+5)
        return urbanization.at[v['Country'],f'{range_low}-{range_high}']
    except:
        return None

In [13]:
def get_distance_equator(v):
    if v['City'] == 'RIC':
        return 5466
    if v['City'] == 'TOR':
        return 4859
    if v['City'] == 'WIN':
        return 4703
    if v['City'] == 'NEW':
        return 4527
    return None

In [14]:
def get_climate(v):
    if v['City'] == 'RIC':
        return 'Cfb'
    if v['City'] == 'TOR':
        return 'Dfb'
    if v['City'] == 'WIN':
        return 'Dfa'
    if v['City'] == 'NEW':
        return 'Cfa'
    return None

# Number of floors
This helper function simply counts the number of floors in a given building.

In [15]:
def number_of_floors(v):
    notna = set()
    for ki,vi in v.items():
        if vi == vi and '_' in ki:
            notna.add(ki.split('_')[0])
    return len(notna)

# Main conversion loop
For each row, map the values in our format to the values in the Heeren & Fishman (2019) format.

If the conversion is 1:1, it is not explained below. Otherwise, a comment explains the motivation.

In [16]:
results = []
for k,v in tqdm(df.iterrows(),total=len(df)):
    output = {}
    output['sid'] = v['Building Identifier']
    output['Country'] = v['Country']
    output['Region'] = v['City']
    output['construction_period_start'] = v['Construction Date']
    output['construction_period_end'] = v['Construction Date']
    
    #We initialize all material values as 0, and then fill them in the next step.
    output['steel'] = 0
    output['copper'] = 0
    output['aluminum'] = 0
    output['unspecified_metal'] = 0
    output['wood'] = 0
    output['paper_cardboard'] = 0
    output['straw'] = 0
    output['concrete'] = 0
    output['cement'] = 0
    output['aggregates'] = 0
    output['brick'] = 0
    output['mortar_plaster'] = 0
    output['mineral_fill'] = 0
    output['plaster_board_gypsum'] = 0
    output['Adobe'] = 0
    output['asphalt'] = 0
    output['bitumen'] = 0
    output['natural_stone'] = 0
    output['cement_asbestos'] = 0
    output['Clay'] = 0
    output['glass'] = 0
    output['siding (unspecified material)'] = 0
    output['Ceramics'] = 0
    output['plastics'] = 0
    output['polystyrene'] = 0
    output['PVC'] = 0
    output['Linoleum'] = 0
    output['Carpet'] = 0
    output['Heraklith'] = 0
    output['Mineral_wool'] = 0
    output['insulation_unspecified'] = 0
    output['other (unspecified material)'] = 0
    
    
    #Using our helper function get_material_name(), we parse the column and find the relevant mapping in the mapper datasheet.
    for l,w in v.items():
        if l not in headings and w==w:
            split = re.split('[_\.\ ]',l)
            out = get_material_name(l)
            if not out:
                continue
            else:
                output[out] += w
    
    output['building_description'] = ""  #We do not have a building description in our database.
    output['no_floors'] = number_of_floors(v)
    output['occupation'] = None #We do not have building occupation in our database.
    output['building_type'] = v['Building Type']
    output['measurement_type'] = 'Case study'
    output['urban_rural'] = 'Urban'
    output['floor_area_type'] = None #We do not have floor area type in our database.
    output['Authors'] = v['Contributor']
    output['publication_year'] = 2021
    output['publication_title'] = 'A Construction Classification System Database for Understanding Resource Use in Building Construction'
    output['publication_outlet'] = None #This will be updated when our submission is accepted.
    output['copyright_publication'] = None #This will be updated when our submission is accepted.
    output['DOI'] = None #This will be updated when our submission is accepted.
    output['URL'] = None #This will be updated when our submission is accepted.
    output['comment_scope'] = None #Not available in our database.
    output['comment_conversion'] = None #Not available in our database.
    output['comment_aggregation'] = 'NA' #Not applicable in our database.
    output['comment_primary'] = 'Material quantities were obtained through performing take offs directly from the building drawings.'
    output['comment_secondary'] = None #Not available in our database.
    output['data_entry'] = None #Not available in our database.
    output['global_region'] = 'NA' #All of our cities are in North America.
    output['climate_classification'] = get_climate(v)
    output['equator_distance'] = get_distance_equator(v)
    output['heating_degree_days_0'] = None #We were not able to obtain this data for a significant number of the years in our database.
    output['heating_degree_days_1'] = None #We were not able to obtain this data for a significant number of the years in our database.
    output['cooling_degree_days_0'] = None #We were not able to obtain this data for a significant number of the years in our database.
    output['cooling_degree_days_1'] = None #We were not able to obtain this data for a significant number of the years in our database.
    
    #As our entries are either in the US or Canada, we provide the values for those two countries explicitly.
    output['country_land_area'] = 9984670 if v['Country'] == 'CA' else 9833517
    
    #We obtain these values from our external datasets.
    output['population_0'] = population[str(int(v['Construction Date']))].values[0] if str(int(v['Construction Date'])) in population.columns else None
    output['population_1'] = output['population_0']
    output['urbanization_0'] = get_urbanization(v)
    output['urbanization_1'] = output['urbanization_0']
    output['gdp_0'] = get_gdp(v)
    output['gdp_1'] = output['gdp_0']
    
    #We were not able to obtain this data for a significant number of the years in our database.
    output['hdi_0'] = None
    output['hdi_1'] = None
    results.append(output)
    
#Convert to a Pandas dataframe.
results = pd.DataFrame(results)
results.set_index('sid',inplace=True)

100%|██████████| 54/54 [00:03<00:00, 14.69it/s]


In [17]:
results.to_csv('converted_dataset.csv')